In [1]:
import pandas as pd
import arviz as az
from function_seasonality import *

In [2]:
train=pd.read_excel(r"../data/chem_train.xlsx")

In [3]:
train=make_periodic(train) 

In [4]:
X_formaldehyd,y_formaldehyd=make_Xy(train,["cos_theta","sin_theta"],"formaldehyd") 
nuts_kernel_formaldehyd,gpr_formaldehyd=model(X_formaldehyd,y_formaldehyd) 
nuts_kernel_formaldehyd,gpr_formaldehyd,mcmc_formaldehyd=train_model(X_formaldehyd,nuts_kernel_formaldehyd,gpr_formaldehyd) 
torch.save(gpr_formaldehyd,"../models/formaldehyd_seasonality");

Sample: 100%|██████████████████████████████████████| 15000/15000 [17:30, 14.28it/s, step size=8.47e-01, acc. prob=0.856]
/data/michaelf/miniconda3/lib/python3.10/site-packages/arviz/data/io_pyro.py:158: UserWarning: Could not get vectorized trace, log_likelihood group will be omitted. Check your model vectorization or set log_likelihood=False
  warnings.warn(


In [5]:
X_acetald,y_acetald=make_Xy(train,["cos_theta","sin_theta"],"acetald") 
nuts_kernel_acetald,gpr_acetald=model(X_acetald,y_acetald) 
nuts_kernel_acetald,gpr_acetald,mcmc_acetald=train_model(X_acetald,nuts_kernel_acetald,gpr_acetald) 
torch.save(gpr_acetald,"../models/acetald_seasonality");

Sample: 100%|██████████████████████████████████████| 15000/15000 [16:38, 15.02it/s, step size=7.52e-01, acc. prob=0.913]
/data/michaelf/miniconda3/lib/python3.10/site-packages/arviz/data/io_pyro.py:158: UserWarning: Could not get vectorized trace, log_likelihood group will be omitted. Check your model vectorization or set log_likelihood=False
  warnings.warn(


In [6]:
X_acetone,y_acetone=make_Xy(train,["cos_theta","sin_theta"],"acetone") 
nuts_kernel_acetone,gpr_acetone=model(X_acetone,y_acetone) 
nuts_kernel_acetone,gpr_acetone,mcmc_acetone=train_model(X_acetone,nuts_kernel_acetone,gpr_acetone) 
torch.save(gpr_acetone,"../models/acetone_seasonality");

Sample: 100%|██████████████████████████████████████| 15000/15000 [16:11, 15.45it/s, step size=8.32e-01, acc. prob=0.897]
/data/michaelf/miniconda3/lib/python3.10/site-packages/arviz/data/io_pyro.py:158: UserWarning: Could not get vectorized trace, log_likelihood group will be omitted. Check your model vectorization or set log_likelihood=False
  warnings.warn(
